In [1]:
import math
import numpy as np
import pandas as pd
from oucp_utils import *
from scipy.optimize import fsolve
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

In [2]:
# carbon content per unit of energy
h1 = 1

# elasticity parameters
sigma = 1
sigmaE = 1
theta = 4
rho = 0
alpha = 0.15

scenario = 'opt'

# elasticity of supply of renewables
epsilonSstar1 = 0.5
epsilonS1 = 0.5

# baseline, no renewable energy
epsilonSvec = [(epsilonS1, h1, 1)]
epsilonSstarvec = [(epsilonSstar1, h1, 1)]
assert(sum(k for i,j,k in epsilonSvec) == 1)
assert(sum(k for i,j,k in epsilonSstarvec) == 1)


data = pd.read_excel("../../raw_data/baselinecarbon_direct_2018.xlsx")
data = pd.read_csv("../../raw_data/BaselineCarbon_2015.csv")
#data['Cey'] = data['Cey'] + data['Ced']
#data['Ceystar'] = data['Ceystar'] + data['Cedstar']
#data['Ced'] = 0
#data['Cedstar'] = 0
data['jxbar']=data['Cex']/(data['Cex'] + data['Ceystar'])
data['jmbar']=data['Cey']/(data['Cey'] + data['Cem'])

if scenario == 'constrained':
    data = data[(data['region_scenario'] == 3)] 
    tax_scenario= ['purete','puretc','puretp','EC_hybrid','EP_hybrid','PC_hybrid','EPC_hybrid']
elif scenario == 'global':
    data = data[(data['region_scenario'] == 3)] 
    tax_scenario= ['global']
elif scenario == 'opt':
    data = data[data['region_scenario'] != 4]
    tax_scenario= ['Unilateral']
#tax_scenario = ['Unilateral']
data = data[(data['region_scenario'] == 3)]
tax_scenario = ['EPC_hybrid']

In [3]:
def max_welfare(p, phi, tax, region_data):
    return -model.comp_welfare(p, phi, tax,region_data)

In [4]:
phi_list = np.arange(0,2,0.1)
model_parameters = (theta, sigma, sigmaE, epsilonSvec, epsilonSstarvec, rho, alpha)
model = taxModel(data, tax_scenario, phi_list, model_parameters)

In [22]:
index, region_data = next(model.data.iterrows())
tax = 'PC_hybrid'
phi = 9.9

In [16]:
res = minimize(max_welfare, [1, 0], args = (phi, tax, region_data), method='nelder-mead', tol=1e-10, options={'maxfev': 1000}, bounds = [(0.01,np.inf), (0, 0)])
res

 final_simplex: (array([[3.57213726, 0.        ],
       [3.57213726, 0.        ],
       [3.57213726, 0.        ]]), array([-35.02207126, -35.02207126, -35.02207126]))
           fun: -35.02207126095468
       message: 'Optimization terminated successfully.'
          nfev: 125
           nit: 65
        status: 0
       success: True
             x: array([3.57213726, 0.        ])

In [23]:
res = minimize(max_welfare, [1, 0], args = (phi, tax, region_data), method='trust-constr', tol=1e-11, bounds = [(0.01,np.inf), (0, 1)])
res

 barrier_parameter: 3.276800000000003e-12
 barrier_tolerance: 3.276800000000003e-12
          cg_niter: 225
      cg_stop_cond: 2
            constr: [array([3.5817335 , 0.77230867])]
       constr_nfev: [0]
       constr_nhev: [0]
       constr_njev: [0]
    constr_penalty: 1.0
  constr_violation: 0.0
    execution_time: 43.392499685287476
               fun: -36.38994086659706
              grad: array([1.73069355e-06, 1.23977661e-05])
               jac: [<2x2 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>]
   lagrangian_grad: array([1.60489069e-06, 5.64422010e-07])
           message: '`xtol` termination condition is satisfied.'
            method: 'tr_interior_point'
              nfev: 456
              nhev: 0
               nit: 141
             niter: 141
              njev: 152
        optimality: 1.6048906857409015e-06
            status: 2
           success: True
         tr_radius: 1.0000000000000006e-12
          

In [24]:
pe = fsolve(model.comp_cons_eq, [1], args = (2, res.x, phi, tax, region_data))[0]
pe

0.7399307759860462

In [25]:
arr = [pe, res.x[0], res.x[1]]
model.comp_all(pe, 2, arr[1:3], phi, tax, region_data)['welfare']

36.38994086659706

In [26]:
model.comp_obj(arr[0], 2, arr[1:3], phi, tax, region_data)

(-7.105427357601002e-13, 7.43483940368606e-6, 1.05814453519315e-5)

In [11]:
res_foc = model.solveOne(phi, tax, region_data, 1,0,0,0)
res_foc

(0.8793545784443226, 3.6839741177666023, 0, 9.9, 1)

In [12]:
pe - res_foc[0]

1.2206121224878075e-08

In [13]:
res.x[0] - res_foc[1]

-3.276121587347802e-07

In [14]:
model.comp_obj(res_foc[0], 2, [res_foc[1],0], phi, tax, region_data)

(1.7763568394002505e-12, -2.52953213930596e-12, 0)

In [14]:
model.comp_all(res_foc[0], 2, [res_foc[1],0], phi, tax, region_data)['welfare']

87.90613059476104

In [14]:
model.comp_obj(1.05739466576942, 2, [0.8465, 0], phi, tax, region_data)

(-0.5452638053042485, 0.606415162319092, 0)

In [16]:
def obj_fn_te(p, phi, tax, region_data):
    diff, diff1 = model.te_obj(p, phi, tax, region_data)
    return abs(diff) + abs(diff1)

def obj_fn(p, phi, tax, region_data):
    diff, diff1, diff2 = model.obj_system(p, phi, tax, region_data)
    return abs(diff) + abs(diff1) + abs(diff2)

In [21]:
model.solveOne(phi,tax,region_data, 1,0,2,1)

did not converge, tax is PC_hybrid region is OECD37 as Home phi is 2 guess is [1, 0, 2]
converged on second try


(0.8566417266416129,
 0.8158650732485632,
 0.5275261066696807,
 0.8158650732485632,
 1)

In [10]:
init_guess = np.random.rand(1000, 2) * 10000
result = []
for pe, tb in init_guess:
    #res = model.solve_obj(phi, tax, region_data, init_guess = [pe,tb, 0], verbose = False, second_try = False)
    res = minimize(obj_fn, [pe,tb], args = (phi, tax, region_data), method='nelder-mead', tol=0.000001, options={'maxfev': 100000})
    result.append((res.x,res.fun, (pe,tb)))

In [13]:
for res in result:
    if not (np.isclose(abs(res[0]), [1.24094625,1.33333333])).all():
        print(res)
    #if res[1] == 1:
    #    print(res)

(array([ 1.09860833, -0.79654207]), 26.688391712445423, (5.699311173572585, 5093.960546141944))


In [15]:
#model.solve_obj(2, tax, region_data, init_guess = [1,0,0], verbose = False)
fsolve(model.te_obj, [1,2], args = (phi, tax, region_data), full_output = True, maxfev = 100000)

(array([1.24094625, 1.33333333]),
 {'nfev': 10,
  'fjac': array([[-0.92146996,  0.38844962],
         [-0.38844962, -0.92146996]]),
  'r': array([-44.30783921, -21.44420869,  30.04099335]),
  'qtf': array([7.19505380e-08, 9.34561708e-08]),
  'fvec': array([-7.13491488e-11, -4.09308143e-11])},
 1,
 'The solution converged.')